# Lbl2Vec Model

## LDA Topic Inference

Unsupervised text classification model. Requires sort of a dataset that can map terms to the correct clusters.
Can be done manually during the training stage of the LDA. But implementing this in the case where it might out-perform human input (such as long topic lists).
There has also been a paper showing that such models can perform better than LDAs; so if effective, the Lbl2vec model can be used for topic modeling directly.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups     # the dataset

In [ ]:
# vars here

In [2]:
# load data
train = fetch_20newsgroups(subset='train', shuffle=False)
test = fetch_20newsgroups(subset='test', shuffle=False)

# parse data to pandas DataFrames
newsgroup_train = pd.DataFrame({'article':train.data, 'class_index':train.target})
newsgroup_test = pd.DataFrame({'article':test.data, 'class_index':test.target})

# load labels with keywords
labels = pd.read_csv('20newsgroups_keywords.csv',sep=';')

In [ ]:
# data pre-processing

# split keywords by separator and save them as array
labels['keywords'] = labels['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels['keywords'] = labels['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['keywords'].apply(lambda row: len(row))

# lets check our keywords
print(labels)